In [ ]:
from metadrive.envs.metadrive_env import MetaDriveEnv
from metadrive.component.map.base_map import BaseMap
from metadrive.component.map.pg_map import MapGenerateMethod
from metadrive.examples.ppo_expert.torch_expert import torch_expert as expert

import logging
from IPython.display import Image, clear_output
import pandas as pd
from pprint import pprint

from PIL import Image
import numpy as np

In [ ]:
def create_env(seed=0):
    # ===== Termination Scheme =====
    termination_sceme = dict(
        out_of_route_done=False,
        on_continuous_line_done=False,
        crash_vehicle_done=True,
        crash_object_done=True,
        crash_human_done=True,
    )
    # ===== Map Config =====
    map_config = {
        BaseMap.GENERATE_TYPE: MapGenerateMethod.BIG_BLOCK_NUM,
        BaseMap.GENERATE_CONFIG: 10,  # 20 block
    }

    cfg = dict(
        # use_render=True,
        horizon=1000,  # figure out what do do aboutit
        record_episode=True,
        start_seed=seed,
        map_config=map_config,
        **termination_sceme
    )
    env = MetaDriveEnv(config=cfg)
    return env

In [ ]:
def run_scenario(seed: int = 0):
    env = create_env(seed)
    _, step_info = env.reset(seed)

    while True:
        # get action from expert driving
        action = expert(env.agent, deterministic=True)
        obs, reward, tm, tr, step_info = env.step(action)

        if tm or tr:
            break

    # save_map_data(env)
    env.close()

In [ ]:
run_scenario(10)

In [ ]:
env = create_env()
obs, step_info = env.reset(0)

In [ ]:
for step in range(5):

    # get action from expert driving, or a dummy action
    action = expert(env.agent, deterministic=True)
    obs, reward, tm, tr, step_info = env.step(action)

    if tm or tr:
        break

In [ ]:
def get_map_img(env):
    map = env.current_map.get_semantic_map(
        env.current_map.get_center_point(),
        size=1024 * 2,
        pixels_per_meter=4,
    )
    map = map.squeeze()  # reduce dimensionality
    map = (map * 255).astype(np.uint8)
    img = Image.fromarray(map)
    return img


get_map_img(env)

In [ ]:
map = env.current_map
map_data = map.get_meta_data()
pprint(map_data["block_sequence"])

In [ ]:
map.show_coordinates()